In [1]:
!pip install sqlparse

In [2]:
import sqlparse
from sqlparse.sql import IdentifierList, Identifier
from sqlparse.tokens import Keyword, DML

def extract_tables(sql):
    tables = []
    parsed = sqlparse.parse(sql)
    for statement in parsed:
        if statement.get_type() != 'CREATE':
            continue
        tokens = statement.tokens
        for i, token in enumerate(tokens):
            if token.ttype is Keyword and token.value.upper() == 'TABLE':
                table_name = tokens[i+2].value.strip('"`')
                columns = []
                parenthesis = [t for t in tokens if t.ttype is None and '(' in str(t)]
                if parenthesis:
                    col_defs = str(parenthesis[0]).strip('()').split(',')
                    for col in col_defs:
                        parts = col.strip().split()
                        if len(parts) >= 2:
                            col_name = parts[0].strip('`')
                            col_type = parts[1]
                            columns.append((col_name, col_type))
                tables.append((table_name, columns))
    return tables

In [4]:
def generate_puml(tables):
    puml = ["@startuml", "skinparam classAttributeIconSize 0"]
    for table_name, columns in tables:
        puml.append(f"class {table_name} {{")
        for col_name, col_type in columns:
            puml.append(f"  {col_name} : {col_type}")
        puml.append("}")
    puml.append("@enduml")
    return '\n'.join(puml)

# Load SQL file
with open('/content/sample_data/dps2.sql', 'r') as f:  # ganti dengan nama file-mu
    sql = f.read()

tables = extract_tables(sql)
puml_code = generate_puml(tables)

# Simpan sebagai file PlantUML
with open('/content/master_data.puml', 'w') as f:
    f.write(puml_code)

print("✅ ERD berhasil dibuat jadi file PlantUML: erd.puml")

✅ ERD berhasil dibuat jadi file PlantUML: erd.puml
